#01. Import Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

#02. Data

In [2]:
path = r'/Users/golde/Instacart Basket Analysis/'

In [3]:
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_updated2.pkl'))

In [4]:
customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [5]:
pd.options.display.max_rows = None

In [6]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


#03. Wrangle Data

In [7]:
#change column names 
customer.rename(columns = {'n_dependants' : 'dependents'}, inplace = True)

In [8]:
customer.rename(columns = {'STATE' : 'state'}, inplace = True)

In [9]:
customer.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [10]:
customer.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

In [11]:
customer.rename(columns = {'Gender' : 'gender'}, inplace = True)

In [12]:
customer.rename(columns = {'Age' : 'age'}, inplace = True)

In [13]:
customer.head()

,user_id,first_name,last_name,gender,state,age,date_joined,dependents,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [14]:
customer.describe()

,user_id,age,dependents,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#04. Consistency Checks

In [15]:
#check for null values
customer.isnull().sum()

user_id            0
first_name     11259
last_name          0
gender             0
state              0
age                0
date_joined        0
dependents         0
fam_status         0
income             0
dtype: int64

First names are not important to the analysis since there is a user_id, I am going to leave them null. 

In [16]:
#check for duplicates
df_dups = customer[customer.duplicated()]

In [17]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,dependents,fam_status,income


In [18]:
#check for mixed types
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

first_name


In [19]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  int64 
 1   first_name   194950 non-null  object
 2   last_name    206209 non-null  object
 3   gender       206209 non-null  object
 4   state        206209 non-null  object
 5   age          206209 non-null  int64 
 6   date_joined  206209 non-null  object
 7   dependents   206209 non-null  int64 
 8   fam_status   206209 non-null  object
 9   income       206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [20]:
customer['first_name'] = customer['first_name'].astype('str')

In [21]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  int64 
 1   first_name   206209 non-null  object
 2   last_name    206209 non-null  object
 3   gender       206209 non-null  object
 4   state        206209 non-null  object
 5   age          206209 non-null  int64 
 6   date_joined  206209 non-null  object
 7   dependents   206209 non-null  int64 
 8   fam_status   206209 non-null  object
 9   income       206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [22]:
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

#05. Combine Data

In [23]:
#combine dataframes
ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,exists,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spender_type,spender_flag,frequency_type,frequency_flag
0,2398795,1,2,3,7,15.0,196,1,1,both,...,both,Regularly busy,2nd slowest day,Average orders,10,New customer,6.372222,Low Spender,20.5,Non-Frequent Customer
1,473747,1,3,3,12,21.0,196,1,1,both,...,both,Regularly busy,2nd slowest day,Average orders,10,New customer,6.372222,Low Spender,20.5,Non-Frequent Customer
2,2254736,1,4,4,7,29.0,196,1,1,both,...,both,Least busy,Least busy,Average orders,10,New customer,6.372222,Low Spender,20.5,Non-Frequent Customer
3,431534,1,5,4,15,28.0,196,1,1,both,...,both,Least busy,Least busy,Average orders,10,New customer,6.372222,Low Spender,20.5,Non-Frequent Customer
4,3367565,1,6,2,7,19.0,196,1,1,both,...,both,Regularly busy,Regularly busy,Average orders,10,New customer,6.372222,Low Spender,20.5,Non-Frequent Customer


In [24]:
#merge
df_merged = ords_prods_merged.merge(customer, on = 'user_id', indicator = 'merged')

In [25]:
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_updated3.pkl'))